<a href="https://colab.research.google.com/github/radicalcoder/ML-Project/blob/master/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import random
import matplotlib
import matplotlib.pyplot as plt
import skimage.data
import skimage.transform
import tensorflow as tf
import numpy as np
import pandas as pd


# Allow image embeding in notebook
%matplotlib inline
!git clone -l -s https://github.com/radicalcoder/ML-Project.git data
%cd data
!ls

Cloning into 'data'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 7373 (delta 13), reused 0 (delta 0), pack-reused 7349
Receiving objects: 100% (7373/7373), 247.13 MiB | 33.48 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Checking out files: 100% (7225/7225), done.
/content/data/data
BelgiumTSC_Testing   MLP_final_initialdraft.ipynb  README.md
BelgiumTSC_Training  MLP.ipynb			   traffic.py


In [0]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    # Get all subdirectories of data_dir. Each represents a label. The list is in arbitrary order.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(data_dir + d)] # There may be .txt files in the data_dir hence we need to check
    directories.sort()
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    #print(directories)
    #print(len(directories))
    labels = []
    images = []
    for d in directories:
        label_dir = data_dir + d
        #print(label_dir)
        file_names = [label_dir + '/' + f
                      for f in os.listdir(label_dir) if f.endswith(".ppm")]
        #print(file_names)
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))   #use parameter as_grey??
            labels.append(int(d))
    return images, labels


# Load training and testing datasets.
train_data_dir = "BelgiumTSC_Training/Training/"
test_data_dir = "BelgiumTSC_Testing/Testing/"

images, labels = load_data(train_data_dir)
#print(labels)

In [38]:
print("Unique Labels: {0}\nTotal Images: {1}".format(len(set(labels)), len(images)))

Unique Labels: 62
Total Images: 4575
